##### Open BolsadeMadrid website with Selenium

In [12]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
import pandas as pd 
import re 

opts = Options()
opts.set_headless()
assert opts.headless  # Operating in headless mode

<ipython-input-12-79c39fba795c>:7: DeprecationWarning: use setter for headless property instead of set_headless
  opts.set_headless()


In [25]:
driver = Firefox(options=opts)
driver.get('https://www.bolsamadrid.es')

##### Process to lacate the Comité asesor members page

In [28]:
driver.title

'Bolsa de Madrid'

In [30]:
# as the menu is in javascript, navigate through the Web Map
# Find and click the Mapa Web link

mapaweb_link = driver.find_element_by_partial_link_text('Mapa').click()
# Check the page title to verify the navigation is correct
driver.title

'Bolsa de Madrid - Mapa web'

In [31]:
#the previous command sometimes fails due to the site coockies 
driver.get('https://www.bolsamadrid.es/esp/aspx/Comun/MapaWeb.aspx')
driver.title

'Bolsa de Madrid - Mapa web'

In [32]:
# Display the content under 'Comité Asesor Técnico'
element = driver.find_element_by_id("ctl00_Contenido_Mapan110")
driver.execute_script("arguments[0].click();", element)

In [11]:
# Click on memebers
miembros_link = driver.find_element_by_id('ctl00_Contenido_Mapat113').click()

In [12]:
driver.title

'Bolsa de Madrid - Miembros'

In [13]:
url=driver.current_url
print(url)

https://www.bolsamadrid.es/esp/SBolsas/Comite-Asesor-Tecnico-Miembros.aspx?1


##### Once in 'Miembros' page, will use BeatifulSoup to get some content

In [ ]:
import requests
from bs4 import BeautifulSoup

page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")
soup

In [15]:
#get the content in the central column where the Members are
div = soup.find("div", {"class": "col5"})

In [ ]:
# Get the list elements with the members information
lili=[]
uls = div.find_all('ul')
for ul in uls:
    newsoup = BeautifulSoup(str(ul), 'html.parser')
    lis = newsoup.find_all('li')
    for li in lis:
        print(li)
        print('..................')
        lili.append(li)
type(lili)

In [ ]:
#clean the html to later extract the information
names=[]
studies=[]
career=[]
for ll in lili:
    ll1=str(ll)
    ll1=ll1.replace('<strong><strong>', '<strong>')
    ll1=ll1.replace('</strong><br/></strong>', '<br/></strong>')
    ll1=ll1.replace('Ph.D.', 'Ph D')
    ll1=ll1.replace('Ph. D.', 'Ph D')
    ll1=ll1.replace('U.', 'U')
    ll1=ll1.replace('&amp;', '&')
    ll1=ll1.replace('País Vasco, Mikel es Catedrático', 'País Vasco. Mikel es Catedrático')
    ll1=ll1.replace('López</strong>', 'López,')
    ll1=ll1.replace('miembro del mismo.', 'miembro del mismo.<br/>')
    
    name=re.findall(r'<strong.*?>(.*?)<br/>', ll1)
    names.append(name)
    print(name)
    edu=re.findall(r'/strong.*?>(.*?)(?=\.)', ll1)
    studies.append(edu)
    print(edu)
    car=re.findall(r'\.(.*?)</li>', ll1)
    career.append(car)
    print(car)
    print('..............\n\n')

##### Finnally, convert the information into a table

In [89]:
career[9]=''

In [90]:
names=[' '.join(strings) for strings in names]
studies=[' '.join(strings) for strings in studies]
career=[' '.join(strings) for strings in career]

In [91]:
df = pd.DataFrame(list(zip(names, studies,career)),
               columns =['Name', 'Studies','Career'])

df

,Name,Studies,Career
0,"Mikel Tapia, en calidad de Presidente del mismo",Licenciado y Doctor en Economía por la Univers...,Mikel es Catedrático de Finanzas en la Univer...
1,Carmen Ansotegui,Carmen es Ph D in Economics por la Universidad...,"Profesora de Finanzas en ESADE, donde ha impa..."
2,Natividad Blasco,Licenciada y Doctora en Ciencias Económicas y ...,"Además de su labor docente e investigadora, f..."
3,Pablo Fernández,"Doctor en Finanzas (Ph D Harvard U), MBA (IESE...",Profesor de Dirección Financiera en el IESE y...
4,Máximo Ferrando Bolado,Licenciado y Doctor en Ciencias Económicas y E...,Ha sido Catedrático de Finanzas Empresariales...
5,Marta Gómez-Puig,Licenciada en Ciencias Económicas y Empresaria...,Actualmente es Catedrática de Universidad en ...
6,Antonio Giralt,Licenciado en Ciencias Económicas y Empresaria...,"Hasta 2016, Presidente del MAB y de LATIBEX, ..."
7,José Ramón Lasuén,Es doctor en ciencias económicas y máster por ...,"Dedicado a la docencia, ha sido profesor auxi..."
8,Simón Sosvilla Rivero,Licenciado en Ciencias Económicas y Empresaria...,Ha sido Investigador senior y subdirector eje...
9,"Carmen López, en calidad de Secretaria del Com...",,
